In [45]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import time
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import sys
import pprint
max_chrome_driver_path = '/Users/maxlien/Downloads/chromedriver_mac_arm64/chromedriver'

from dotenv import load_dotenv
import os

def config():
    load_dotenv()
  
config()

username = str(os.getenv('MONOPLE_GMAIL'))
password=str(os.getenv('MONOPLE_PASSWORD'))

In [48]:
class StackExchangeScraper:
    def __init__(self, chrome_driver_path=max_chrome_driver_path,username=username, password=password):
        self.driver_path = chrome_driver_path
        self.options = webdriver.ChromeOptions()
        #print(self.options)
        self.driver = webdriver.Chrome(ChromeDriverManager().install())#,options=self.options)#executable_path=self.driver_path)
        self.wait = WebDriverWait(self.driver, 10)

        self.username =username
        self.password = password

    # ... rest of the class methods remain unchanged ...

# Note: Only the initialization method of the class is shown here for brevity.
# The rest of the class methods would remain unchanged from the previous version.
    def _set_up_agent(self):
        #set useragent
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    def _setup_headless_driver(self):
        """
        Sets up a headless Selenium Chrome driver.
        """
        self.options.add_argument('--headless')
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(self.driver_path, options=options)
    def _wait(self,type, name):
        return self.wait.until(
                EC.presence_of_element_located((type, name))
            )
    def _login(self, username, password):
        
        
        login_button_XPATH = '/html/body/div[1]/header[1]/div/nav/ol/li[4]/a'
        login_button = self._wait(By.XPATH, login_button_XPATH)
        self.driver.execute_script("arguments[0].scrollIntoView();", login_button)
        login_button.click()


        
    

        username_box = self._wait(By.XPATH, '//*[@id="email"]')
        username_box.send_keys(username)

        password_box = self._wait(By.XPATH,'//*[@id="password"]' )
        password_box.send_keys(password)


        submit_button =self._wait(By.XPATH, '//*[@id="submit-button"]') 
        self.driver.execute_script("arguments[0].scrollIntoView();", submit_button)
        submit_button.click()




    def search_elements(self, input):
        self.driver.get('https://stackexchange.com')

        self._login(self.username,self.password)

        search_box = self._wait(By.XPATH,'//*[@id="search"]/div/input')
        search_box.send_keys(input)
        search_box.send_keys(Keys.ENTER)

         # Using the new provided XPath to find the desired element
        try:
            element = self._wait(By.CLASS_NAME, 'result-link')
        except TimeoutException:
            print("Element not found or page took too long to load")
            return []
        
        return [element]

    def scrape(self, input):
        """
        Main method to scrape StackExchange for a given query and return the texts in a list.
        """
        url = 'https://stackexchange.com'
        #self._setup_headless_driver()
        
        elements = self.search_elements(input)
        texts = [i.text for i in elements if i.text != '']
        time.sleep(10)
        #self.driver.quit()
        return texts

# Note: As before, the Selenium portion cannot be executed in this environment. 
# The code should be run in a local environment with the appropriate browser driver.


In [49]:
qs = StackExchangeScraper(chrome_driver_path=max_chrome_driver_path,username=username,password=password)
qs.scrape('what is periodic table')

AttributeError: 'str' object has no attribute 'capabilities'